# 02. First Steps with LangChain

## 安装依赖

In [6]:
%uv pip install langchain~=1.0 langchain-core~=1.0 langchain-openai~=1.0 langgraph~=1.0

Audited 4 packages in 2ms
Note: you may need to restart the kernel to use updated packages.


In [5]:
%uv pip install python-dotenv~=1.1

Resolved 1 package in 380ms                                          
⠙ Preparing packages... (0/1)                                                   
⠙ Preparing packages... (0/1)--------------     0 B/20.73 KiB           
⠙ Preparing packages... (0/1)---------- 14.88 KiB/20.73 KiB         
⠙ Preparing packages... (0/1)---------- 20.73 KiB/20.73 KiB         
Prepared 1 package in 173ms                                                  
░░░░░░░░░░░░░░░░░░░░ [0/1] Installing wheels...                                 warning: Failed to hardlink files; falling back to full copy. This may lead to degraded performance.
         If the cache and target directories are on different filesystems, hardlinking may not be supported.
         If this is intentional, set `export UV_LINK_MODE=copy` or use `--link-mode=copy` to suppress this warning.
Installed 1 package in 14ms                                 
 + python-dotenv==1.2.1
Note: you may need to restart the kernel to use updated packages.


In [7]:
%uv pip show langchain-core

Name: langchain-core
Version: 1.0.1
Location: /github.com/sammyne/generative-ai-with-lang-chain-2ed/chapter02/.venv/lib/python3.12/site-packages
Requires: jsonpatch, langsmith, packaging, pydantic, pyyaml, tenacity, typing-extensions
Required-by: langchain, langchain-openai, langgraph, langgraph-checkpoint, langgraph-prebuilt
Note: you may need to restart the kernel to use updated packages.


工具类

In [2]:
import os

import dotenv
from langchain_openai import ChatOpenAI


class Config:
    def __init__(self):
        # By default, load_dotenv doesn't override existing environment variables and looks for a .env file in same directory as python script or searches for it incrementally higher up.
        dotenv_path = dotenv.find_dotenv(usecwd=True)
        if not dotenv_path:
            raise ValueError("No .env file found")
        dotenv.load_dotenv(dotenv_path=dotenv_path)

        api_key = os.getenv("OPENAI_API_KEY")
        if not api_key:
            raise ValueError("OPENAI_API_KEY is not set")

        base_url = os.getenv("OPENAI_API_BASE_URL")
        if not base_url:
            raise ValueError("OPENAI_API_BASE_URL is not set")

        model = os.getenv("OPENAI_MODEL")
        if not model:
            raise ValueError("OPENAI_MODEL is not set")

        vl_model = os.getenv("OPENAI_VL_MODEL")

        self.api_key = api_key
        self.base_url = base_url
        self.model = model
        self.vl_model = vl_model

    def new_openai_like(self, **kwargs) -> ChatOpenAI:
        # 参考：https://bailian.console.aliyun.com/?tab=api#/api/?type=model&url=2587654
        # 参考：https://help.aliyun.com/zh/model-studio/models
        # ChatOpenAI 文档参考：https://python.langchain.com/api_reference/openai/chat_models/langchain_openai.chat_models.base.ChatOpenAI.html#langchain_openai.chat_models.base.ChatOpenAI
        return ChatOpenAI(
            api_key=self.api_key, base_url=self.base_url, model=self.model, **kwargs
        )

    def new_openai_like_coder(self, **kwargs) -> ChatOpenAI:
        model = os.environ["OPENAI_CODER_MODEL"]
        # 参考：https://bailian.console.aliyun.com/?tab=api#/api/?type=model&url=2587654
        # 参考：https://help.aliyun.com/zh/model-studio/models
        # ChatOpenAI 文档参考：https://python.langchain.com/api_reference/openai/chat_models/langchain_openai.chat_models.base.ChatOpenAI.html#langchain_openai.chat_models.base.ChatOpenAI
        return ChatOpenAI(
            api_key=self.api_key, base_url=self.base_url, model=model, **kwargs
        )

    def new_openai_like_vl(self, **kwargs) -> ChatOpenAI:
        if not self.vl_model:
            raise ValueError("OPENAI_VL_MODEL is not set")

        # 参考：https://bailian.console.aliyun.com/?tab=api#/api/?type=model&url=2587654
        # 参考：https://help.aliyun.com/zh/model-studio/models
        # ChatOpenAI 文档参考：https://python.langchain.com/api_reference/openai/chat_models/langchain_openai.chat_models.base.ChatOpenAI.html#langchain_openai.chat_models.base.ChatOpenAI
        return ChatOpenAI(
            api_key=self.api_key, base_url=self.base_url, model=self.vl_model, **kwargs
        )

## Exploring LangChain’s building blocks
### Model interfaces
#### LLM interaction patterns

In [9]:
# initialize OpenAI-like model
llm = Config().new_openai_like()

# Both can be used with the same interface
response = llm.invoke("Tell me a joke about light bulbs!")
print(response.content)

Sure! Here's a classic with a twist:

**How many programmers does it take to change a light bulb?**  
*None—it’s a hardware problem!* 💡

Or if you prefer a punny one:  
**Why did the light bulb go to therapy?**  
*Because it had deep-seated issues… and it was feeling a little dim!* 😄

Want another? I’ve got watts more!


#### Development testing (TODO: 等待 langchain-community v1.0.0 正式发布)

In [ ]:
# from langchain_community.llms import FakeListLLM

# # Create a fake LLM that always returns the same responses
# fake_llm = FakeListLLM(responses=["Hello"])

# result = fake_llm.invoke("Any input will return Hello")
# print(result)  # Output: Hello

Hello


#### Working with chat models

In [10]:
from langchain_core.messages import HumanMessage, SystemMessage

# initialize OpenAI-like model
llm = Config().new_openai_like_coder()

messages = [
    SystemMessage(content="You're a helpful programming assistant"),
    HumanMessage(content="Write a Python function to calculate factorial"),
]
response = llm.invoke(messages)
print(response.content)

Here are a few different ways to write a factorial function in Python:

## Method 1: Recursive approach
```python
def factorial_recursive(n):
    """
    Calculate factorial using recursion.
    
    Args:
        n (int): Non-negative integer
    
    Returns:
        int: Factorial of n
    """
    if n < 0:
        raise ValueError("Factorial is not defined for negative numbers")
    if n == 0 or n == 1:
        return 1
    return n * factorial_recursive(n - 1)
```

## Method 2: Iterative approach (recommended)
```python
def factorial_iterative(n):
    """
    Calculate factorial using iteration.
    
    Args:
        n (int): Non-negative integer
    
    Returns:
        int: Factorial of n
    """
    if n < 0:
        raise ValueError("Factorial is not defined for negative numbers")
    
    result = 1
    for i in range(2, n + 1):
        result *= i
    return result
```

## Method 3: Using math.factorial (built-in)
```python
import math

def factorial_builtin(n):
    """
  

#### Reasoning models

In [11]:
from langchain_core.prompts import ChatPromptTemplate


# initialize OpenAI-like model with reasoning_effort parameter
llm = Config().new_openai_like_coder(
    reasoning_effort="high"  # Options: "low", "medium", "high"
)

template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are an experienced programmer and mathematical analyst."),
        ("user", "{problem}"),
    ]
)

chain = template | llm

problem = """
Design an algorithm to find the kth largest element in an unsorted array
with the optimal time complexity. Analyze the time and space complexity
of your solution and explain why it's optimal.
"""
response = chain.invoke({"problem": problem})
print(response.content)

I'll design an algorithm to find the kth largest element using the QuickSelect algorithm, which provides optimal average-case performance.

## Algorithm: QuickSelect (Partition-based Selection)

```python
def find_kth_largest(nums, k):
    """
    Find the kth largest element in an unsorted array.
    
    Args:
        nums: List of integers
        k: Integer representing kth largest position (1-indexed)
    
    Returns:
        The kth largest element
    """
    def quickselect(left, right, k_smallest):
        """
        Find the k_smallest-th smallest element (0-indexed).
        """
        if left == right:
            return nums[left]
        
        # Choose a random pivot to avoid worst-case on sorted arrays
        import random
        pivot_index = random.randint(left, right)
        pivot_index = partition(left, right, pivot_index)
        
        if k_smallest == pivot_index:
            return nums[k_smallest]
        elif k_smallest < pivot_index:
            ret

### Prompts and templates

In [12]:
from langchain_core.prompts import ChatPromptTemplate


chat = Config().new_openai_like()

template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are an English to French translator."),
        ("user", "Translate this to French: {text}"),
    ]
)

formatted_messages = template.format_messages(text="Hello, how are you?")
result = chat.invoke(formatted_messages)
print(result.content)

Bonjour, comment allez-vous ?


### LangChain Expression Language (LCEL)

#### Simple workflows with LCEL

In [3]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate

# Create components
prompt = PromptTemplate.from_template("Tell me a joke about {topic}")

llm = Config().new_openai_like()

output_parser = StrOutputParser()

# Chain them together using LCEL
chain = prompt | llm | output_parser

# Use the chain
result = chain.invoke({"topic": "programming"})
print(result)

Why do programmers prefer dark mode?

Because light attracts bugs! 🐛💻


#### Complex chain example

In [4]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate


chat = Config().new_openai_like()

# First chain generates a story
story_prompt = PromptTemplate.from_template("Write a short story about {topic}")
story_chain = story_prompt | chat | StrOutputParser()

# Second chain analyzes the story
analysis_prompt = PromptTemplate.from_template(
    "Analyze the following story's mood:\n{story}"
)
analysis_chain = analysis_prompt | chat | StrOutputParser()

output_prompt = PromptTemplate.from_template(
    "Here's the story: \n{story}\n\nHere's the mood: \n{mood}"
)
# Combine chains
story_with_analysis = story_chain | analysis_chain

# Run the combined chain
result = story_with_analysis.invoke({"topic": "a rainy day"})
print(result)

The mood of this story evolves significantly, moving from **melancholy and introspection** to **quiet hope and renewal**, ultimately settling into a tone of **peaceful acceptance and gentle optimism**.

### Initial Mood: Melancholy and Isolation  
The opening establishes a somber, almost oppressive atmosphere. The rain doesn’t “fall” but “settles”—a verb that conveys weight, stagnation, and quiet suffocation. Descriptions like “cool, grey wool blanket wrapped tight around the world,” “untouched coffee,” and Arthur’s sense of being “dissolved” evoke emotional numbness, loneliness, and resignation. The world outside is muted—“watercolor blur,” “muffled symphony,” “dark mirrors”—reinforcing a sense of detachment and internal stagnation. Arthur is physically present but emotionally adrift, viewing life as a series of disconnected disappointments.

### Turning Point: Sudden Joy and Contrast  
The arrival of the child in the yellow raincoat introduces a dramatic shift. Her laughter is descri

In [5]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough


chat = Config().new_openai_like()

# First chain generates a story
story_prompt = PromptTemplate.from_template("Write a short story about {topic}")
story_chain = story_prompt | chat | StrOutputParser()

# Second chain analyzes the story
analysis_prompt = PromptTemplate.from_template(
    "Analyze the following story's mood:\n{story}"
)
analysis_chain = analysis_prompt | chat | StrOutputParser()

output_prompt = PromptTemplate.from_template(
    "Here's the story: \n{story}\n\nHere's the mood: \n{mood}"
)

# Using RunnablePassthrough.assign to preserve data
enhanced_chain = RunnablePassthrough.assign(
    story=story_chain  # Add 'story' key with generated content
).assign(
    analysis=analysis_chain  # Add 'analysis' key with analysis of the story
)

result = enhanced_chain.invoke({"topic": "a rainy day"})
print(result.keys())  # Output: dict_keys(['topic', 'story', 'analysis'])
# dict_keys(['topic', 'story', 'analysis'])

dict_keys(['topic', 'story', 'analysis'])


In [6]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from operator import itemgetter

chat = Config().new_openai_like()

# First chain generates a story
story_prompt = PromptTemplate.from_template("Write a short story about {topic}")
story_chain = story_prompt | chat | StrOutputParser()

# Second chain analyzes the story
analysis_prompt = PromptTemplate.from_template(
    "Analyze the following story's mood:\n{story}"
)
analysis_chain = analysis_prompt | chat | StrOutputParser()

output_prompt = PromptTemplate.from_template(
    "Here's the story: \n{story}\n\nHere's the mood: \n{mood}"
)

# Alternative approach using dictionary construction
manual_chain = (
    RunnablePassthrough()  # Pass through input
    | {
        "story": story_chain,  # Add story result
        "topic": itemgetter("topic"),  # Preserve original topic
    }
    | RunnablePassthrough().assign(  # Add analysis based on story
        analysis=analysis_chain
    )
)
result = manual_chain.invoke({"topic": "a rainy day"})
print(result.keys())  # Output: dict_keys(['story', 'topic', 'analysis'])

dict_keys(['story', 'topic', 'analysis'])


## Running local models

### Getting started with Ollama

In [ ]:
!bash ollama.sh up

In [ ]:
!bash ollama.sh run deepseek-r1:1.5b

In [8]:
%uv pip install langchain-ollama~=1.0

Audited 1 package in 2ms
Note: you may need to restart the kernel to use updated packages.


In [9]:
from langchain_ollama import ChatOllama
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Initialize Ollama with your chosen model
local_llm = ChatOllama(
    model="deepseek-r1:1.5b",
    temperature=0,
    # 将 base_url 的地址部分替换为上一步打印出来的地址。
    base_url="http://172.17.0.3:11434",
)
# Create an LCEL chain using the local model
prompt = PromptTemplate.from_template("Explain {concept} in simple terms")
local_chain = prompt | local_llm | StrOutputParser()
# Use the chain with your local model
result = local_chain.invoke({"concept": "quantum computing"})
print(result)

<think>
Okay, so I need to explain quantum computing in simple terms. Hmm, where do I start? I remember from school that computers use bits, which are either 0 or 1. But wait, quantum computing uses something called qubits instead of regular bits. Maybe it's like a superposition thingy?

Let me think... Quantum bits can be both 0 and 1 at the same time because they're in a state of superposition. That means when you measure them, you don't get just one result; there's a chance they could be 0 or 1 simultaneously. So that's different from classical computers where each bit is either 0 or 1.

But how does this help with computing? Oh right, quantum computers can process multiple possibilities at once because of superposition and something called entanglement. Entanglement means that the state of one qubit is linked to another, so changing one affects the other instantly. That could mean solving problems faster by considering all possible solutions simultaneously.

Wait, but how does this

In [ ]:
!bash ollama.sh down

### Working with Hugging Face models locally

In [10]:
%uv pip install langchain-huggingface~=1.0 transformers~=4.57 torch==2.8.0

Resolved 58 packages in 377ms                                        
⠙ Preparing packages... (0/1)                                                   
⠙ Preparing packages... (0/1)---------------     0 B/26.85 KiB          
⠙ Preparing packages... (0/1)------------ 16.00 KiB/26.85 KiB        
⠙ Preparing packages... (0/1)----------- 26.85 KiB/26.85 KiB        
Prepared 1 package in 173ms                                                  
░░░░░░░░░░░░░░░░░░░░ [0/31] Installing wheels...                                warning: Failed to hardlink files; falling back to full copy. This may lead to degraded performance.
         If the cache and target directories are on different filesystems, hardlinking may not be supported.
         If this is intentional, set `export UV_LINK_MODE=copy` or use `--link-mode=copy` to suppress this warning.
Installed 31 packages in 26.24s                             
 + filelock==3.20.0
 + fsspec==2025.9.0
 + hf-xet==1.2.0
 + huggingface-hub==0.36.0
 + jinja

In [11]:
from langchain_core.messages import SystemMessage, HumanMessage
from langchain_huggingface import ChatHuggingFace, HuggingFacePipeline

# Create a pipeline with a small model:
llm = HuggingFacePipeline.from_model_id(
    model_id="TinyLlama/TinyLlama-1.1B-Chat-v1.0",
    task="text-generation",
    pipeline_kwargs=dict(
        max_new_tokens=512,
        do_sample=False,
        repetition_penalty=1.03,
    ),
)

chat_model = ChatHuggingFace(llm=llm)

# Use it like any other LangChain LLM
messages = [
    SystemMessage(content="You're a helpful assistant"),
    HumanMessage(content="Explain the concept of machine learning in simple terms"),
]
ai_msg = chat_model.invoke(messages)
print(ai_msg.content)

/github.com/sammyne/generative-ai-with-lang-chain-2ed/chapter02/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Device set to use cpu
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


<|system|>
You're a helpful assistant</s>
<|user|>
Explain the concept of machine learning in simple terms</s>
<|assistant|>
Machine learning is a field that involves the use of algorithms and data to learn from and improve upon the behavior of machines or systems. It is a powerful tool for automating tasks and improving the performance of systems by learning from data and making predictions based on that data.

In simple terms, machine learning is the process of using algorithms to analyze and learn from data. This process involves collecting large amounts of data, training the algorithm to identify patterns and relationships in that data, and then using that knowledge to make predictions or make decisions.

For example, a machine learning algorithm might be used to predict the likelihood of a customer purchasing a particular product based on their previous purchase history and other factors. Or it might be used to recommend products to a user based on their browsing and purchase hist

### Tips for local models

In [12]:
import time


def safe_model_call(llm, prompt, max_retries=2):
    """Safely call a local model with retry logic and graceful failure"""
    retries = 0
    while retries <= max_retries:
        try:
            return llm.invoke(prompt)
        except RuntimeError as e:
            # Common error with local models when running out of VRAM
            if "CUDA out of memory" in str(e):
                print(
                    f"GPU memory error, waiting and retrying ({retries+1}/{max_retries+1})"
                )
                time.sleep(2)  # Give system time to free resources
                retries += 1
            else:
                print(f"Runtime error: {e}")
                return "An error occurred while processing your request."
        except Exception as e:
            print(f"Unexpected error calling model: {e}")
            return "An error occurred while processing your request."
    # If we exhausted retries
    return "Model is currently experiencing high load. Please try again later."

In [13]:
# Use the safety wrapper in your LCEL chain
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableLambda

llm = Config().new_openai_like()

prompt = PromptTemplate.from_template("Explain {concept} in simple terms")
safe_llm = RunnableLambda(lambda x: safe_model_call(llm, x))
safe_chain = prompt | safe_llm
response = safe_chain.invoke({"concept": "quantum computing"})
print(response)

content='Sure! Let’s break down quantum computing in simple terms:\n\n### What is a regular (classical) computer?\nA regular computer—like your laptop or phone—uses **bits** to process information. A bit can be either a **0** or a **1**. Everything you do on a computer—watching videos, sending messages, playing games—is built from combinations of these 0s and 1s.\n\n### What’s different about a quantum computer?\nA quantum computer uses **quantum bits**, or **qubits**. Qubits are special because they can be **0, 1, or both at the same time**! This is thanks to a weird but real feature of quantum physics called **superposition**.\n\n#### Think of it like this:\n- A regular bit is like a light switch: it’s either **ON (1)** or **OFF (0)**.\n- A qubit is like a spinning coin: while it’s spinning, it’s kind of **both heads and tails at once**. Only when you stop it (measure it) does it “choose” one side.\n\n### Another quantum trick: Entanglement\nQubits can also be **entangled**. That mea

## Multimodal AI applications
### Text-to-image
#### Using DALL-E through OpenAI
TODO：找到或实现支持 langchain 接口的千义通问文生图模型（候选 qwen-image-plus）。

### Image understanding
#### Using Qwen-VL (similar to Gemini 1.5 Pro)

In [14]:
import base64

from langchain_core.messages.human import HumanMessage

with open("static/stable-diffusion.png", "rb") as image_file:
    image_bytes = image_file.read()
    base64_bytes = base64.b64encode(image_bytes).decode("utf-8")


prompt = [
    {"type": "text", "text": "Describe the image: "},
    {
        "type": "image_url",
        "image_url": {"url": f"data:image/jpeg;base64,{base64_bytes}"},
    },
]


# initialize OpenAI-like model
llm = Config().new_openai_like_vl()
response = llm.invoke([HumanMessage(content=prompt)])
print(response.content)

This is a highly detailed, futuristic digital illustration of a humanoid robot or android, presented in a side-profile view against a high-tech interface background.

**The Robot:**
- **Design:** The robot has a sleek, feminine form with smooth, glossy white armor plating over its head, shoulders, and chest. The design is elegant yet mechanical.
- **Head:** Its face is dark, almost black, with glowing golden-yellow eyes that give it an intense, intelligent gaze. The helmet is partially open on the sides, revealing complex internal circuitry and glowing orange lights within the neck and jaw area.
- **Exposed Mechanics:** The neck and upper torso are partially exposed, showcasing intricate networks of wires, tubes, and glowing components, suggesting advanced internal systems. Small, warm orange lights dot the joints and seams of the armor.
- **Shoulders & Chest:** The shoulder plates are large and curved, while the chest plate is broad and segmented, all contributing to a powerful yet gr

In [15]:
import base64

from langchain_core.messages import HumanMessage


def analyze_image(image_path: str, question: str) -> str:
    chat = Config().new_openai_like_vl(max_tokens=256)

    with open(image_path, "rb") as image_file:
        image_bytes = image_file.read()
        base64_bytes = base64.b64encode(image_bytes).decode("utf-8")

    message = HumanMessage(
        content=[
            {"type": "text", "text": question},
            {
                "type": "image_url",
                "image_url": {
                    "url": f"data:image/jpeg;base64,{base64_bytes}",
                    "detail": "auto",
                },
            },
        ]
    )

    response = chat.invoke([message])
    return response.content


# Example usage
image_path = "static/skyscrapers.png"
questions = [
    "What objects do you see in this image?",
    "What is the overall mood or atmosphere?",
    "Are there any people in the image?",
]

for question in questions:
    print(f"\nQ: {question}")
    print(f"A: {analyze_image(image_path, question)}")


Q: What objects do you see in this image?
A: Based on the image provided, here are the objects and key visual elements I can identify:

-   **Futuristic Cityscape:** The dominant feature is a dense skyline of very tall, modern skyscrapers. Their design is sleek and geometric, with many appearing as glass or reflective structures.
-   **Sun / Light Source:** A bright, stylized sun is positioned in the upper center of the sky, casting a strong glare and lens flare effect.
-   **Sky:** The background is a clear, vibrant blue sky, suggesting either dawn or dusk due to the intense light.
-   **Road / Highway:** In the foreground, there is a multi-lane road or highway stretching towards the horizon and the city.
-   **Light Trails (Motion Blur):** The most prominent feature on the road is long, streaking lines of light. These represent the motion blur of vehicle lights captured over time:
    -   **Red Streaks:** Likely from the taillights of cars moving away from the viewer.
    -   **Whit